<a href="https://colab.research.google.com/github/JTO30/class/blob/master/b215147_%EC%98%A4%EC%A0%95%ED%83%9D_fulladder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as n
import matplotlib


# -----------define functions---------------------
def w_init(a, b):
    w = n.ones((a, b))
    for i in range(a):          # a*b 행렬에 랜덤가중치 초기화
        for j in range(1, b):
            w[i][j] = n.random.normal(0, 2)

    print(w)
    return w


def sig(a):
    return 1 / (1 + n.exp(-a))  # 시그모이드 함수


def sigdir(a):                  #시그모이드 함수의 도함수
    return sig(a) * (1 - sig(a))


def step(a):                     #스텝함수
    return n.array(a > 0, dtype=int)


def relu(a):                    #relu 함수
    return n.maximum(0, a)


def sns(input, weight, bias=0): # 레이어 노드들의 가중합을 구하는 함수. 다음 계산에 필요한 바이어스가 있다면 바이어스 노드도 추가.
    if bias != 0:
        return n.hstack((bias, n.matmul(input, weight.T)))
    else:
        return n.matmul(input, weight.T)


# -----------------------initialize---------------------------


truth_table = n.array([[0, 0, 0, 0, 0],
                       [0, 0, 1, 1, 0],
                       [0, 1, 0, 1, 0],
                       [0, 1, 1, 0, 1],     #fulladder 진리표.
                       [1, 0, 0, 1, 0],
                       [1, 0, 1, 0, 1],
                       [1, 1, 0, 0, 1],
                       [1, 1, 1, 1, 1]])

x = n.ones((8, 100, 4))
x = -x                      # 데이터 배열과 레이블 배열을 나누어 쓸 것입니다.
y = n.zeros((8, 2))         # 데이터세트는 3개의 입력과 x0=-1을 포함하여 총 4개의 데이터로 구성합니다.


for k in range(8):
    for i in range(100):
        for j in range(1, 4):                   # 1과 -1을 중심으로하는 가우시안 난수입력. 각 100개씩 사용합니다.
            if truth_table[k][j - 1] == 1:
                x[k][i][j] = n.random.normal(1, 0.2)
            else:
                x[k][i][j] = n.random.normal(-1, 0.2)
    for j in range(2):
        y[k][j] = truth_table[k][j + 3]         # 에러비교를 위한 데이터레이블


w0 = w_init(30, x.shape[2])
w1 = w_init(30, w0.shape[0] + 1)        # 가중치를 무작위값을 가진 배열로 초기화합니다. 행의 수는 각 레이어의 노드수와 같습니다. 행의 수를 바꿈으로서 노드수를 조작할 수 있습니다.
w2 = w_init(2, w1.shape[0] + 1)


bias_l1 = n.array([-1], dtype=float)
bias_l2 = n.array([-1], dtype=float)

learn = float(0.5)                  #learning rate

delta_w2 = n.zeros((w2.shape[0]))
delta_w1 = n.zeros((w1.shape[0]))   #backpropagation의 계산에 사용할 델타함수를 초기화합니다. 각 노드 마다 값을 갖습니다.
delta_w0 = n.zeros((w0.shape[0]))

print(x)
# ------------fowardpropagation--------------------------------
for c in range(20):
    for v in range(20):
        for m in range(8):
            hdn_l1 = sns(x[m][v], w0, -1)   #첫번째 히든레이어의 가중합을 계산합니다.
            act_l1 = sig(hdn_l1)            #첫번째 히든레이어의 노드값을 계산합니다.

            hdn_l2 = sns(act_l1, w1, -1)
            act_l2 = sig(hdn_l2)

            hdn_l3 = sns(act_l2, w2)        #출력레이어의 가중합을 계산합니다.
            act_l3 = sig(hdn_l3)            #출력레이어의 노드값을 계산합니다.


# ----------------backpropagation------------------------
            print('--------------------------------\n', 'c', c, 'v', v, 'm', m)
            # dE/dw2에 체인룰을 적용해 w2를 갱신합니다.

            for a in range(w2.shape[0]):            # y = act_l3 이고 de/dw2 = (dE/dy0 * dy0/dhdn_l3) * dhdn_l3/w2 중 괄호의 식을 y0,다른 표현으로 act_l3[0]의 델타함수라고 합니다.
                delta_w2[a] = (act_l3[a] - y[m][a]) * sigdir(act_l3[a])

            for a in range(w2.shape[0]):
                for b in range(w2.shape[1]):        # 갱신하려는 가중치에 붙어있던 입력, 가령 이전레이어의 첫번째 노드에서 온 입력이라면 act_l2[0]에, 델타함수, 러닝레이트를 곱한 값이 갱신값이 되겠습니다.
                    w2[a][b] = w2[a][b] - learn * delta_w2[a] * act_l2[b]


                #두번째 히든레이어의 갱신하기

            for a in range(w1.shape[0] - 1):
                for b in range(w2.shape[0]):        #  델타함수 구하기. 앞선 과정과 유사하나, 히든레이어에 입력된 가중합은 다음 레이어의 모든 노드에 신호를 보내기 때문에 다음 노드들의 변화량을 합산해주는 과정이 필요합니다.
                    delta_w1[a] += delta_w2[b] * w2[b][a + 1]   # dE/w1 = dE/hdn_l2 * dhdn_l2/dw1 = sigma(de/dhdn_l3) * dhdn_l3/dhdn_l2 * dhdn_l2/dw1
                delta_w1[a] *= sigdir(act_l2[a + 1])    # 또한 현재 레이어의 바이어스는 이전 레이어에서 입력이 없기 때문에 델타함수 또한 없습니다.

            for a in range(w1.shape[0] - 1):
                for b in range(w1.shape[1]):
                    w1[a][b] = w1[a][b] - learn * delta_w1[a] * act_l1[b]

                #첫번째 히든레이어 갱신하기

            for a in range(w0.shape[0] - 1):
                for b in range(w1.shape[0]):
                    delta_w0[a] += delta_w1[b] * w1[b][a + 1]
                delta_w0[a] *= sigdir(act_l1[a + 1])

            for a in range(w0.shape[0] - 1):
                for b in range(x.shape[2]):
                    w0[a][b] = w0[a][b] - learn * delta_w0[a] * x[m][v][b]

            delta_w2 = n.zeros((w2.shape[0]))   #델타함수 초기화
            delta_w1 = n.zeros((w1.shape[0]))
            delta_w0 = n.zeros((w0.shape[0]))

            out = act_l3
            print(out)
            print(n.around(out))
            print('x\n', x[m][v])

m = 100
chk = n.ones((m, 8, 4))
chk = -chk                  #갱신이 종료된 w에 새로운 데이터를 이용하여 w 검증하기
rslt = n.zeros((m, 8, 2))
err = n.array([0])
score = n.array([0])

for k in range(m):
    for i in range(8):          # 검증데이터 생성. 트레이닝셋과 같은 규모를 사용했습니다.
        for j in range(1, 4):
            if truth_table[i][j - 1] == 1:
                chk[k][i][j] = n.random.normal(1, 0.2)
            else:
                chk[k][i][j] = n.random.normal(-1, 0.2)

for i in range(m):
    for j in range(8):
        hdn_l1 = sns(chk[i][j], w0, bias_l1)
        act_l1 = sig(hdn_l1)
        hdn_l2 = sns(act_l1, w1, bias_l2)
        act_l2 = sig(hdn_l2)
        hdn_l3 = sns(act_l2, w2)
        act_l3 = sig(hdn_l3)
        rslt[i][j] = act_l3
        err += (n.abs(y[j][0] - (act_l3[0])) > 0.5)     # 반올림을 기준으로 에러여부를 판단합니다.
        err += (n.abs(y[j][1] - (act_l3[1])) > 0.5)     # 진리표 1개, 총 8개의 논리값이 단 하나도 틀리지 않은 경우를 성공 단위로 하여 평가를 할 것입니다.
    print(i,'th data set: err number',err)              # 100개의 데이터가 입력되므로 만점은 100점
    if err == 0 :
        score += 1
    err = 0
print(score)
print(rslt[0])      # 0번째 검증데이터의 진리표